# Clusteranalyse

In diesem Notebook wird eine Clusteranalyse über den gesamten Datensatz, noch ohne Feature Engineering, durchgeführt. Ziel ist es, die Städte im Datensatz aufgrund ihrer Schadstoffbelastung in verschiedene Cluster einzuteilen und diese Cluster zu beschreiben.

Die Clusteranalyse verläuft rekursiv und beinhaltet auch die Identifikation und Entfernung von Städten mit extremen (realistischen und unrealistischen) Schadstoffprofilen.

Verwendet werden:
- geodatasets und geopandas für geographische Karten und Plots
- matplotlib.patches zur Individualisierung von Farbkodierungen in Plots
- sklearn für Skalierung, Clustering (K-Means) und PCA

📌 **Datenstand:** `cleaned_air_quality_data_2025-03-27.csv`  
📁 **Importiert aus:** lokaler Datei (--> gitignore)


## 📚 Inhaltsverzeichnis 
(Diese Art von Inhaltsverzeichnis mit Link funktioniert leider in Notebooks nicht, weil die as JSON gespeichert werden und nicht als HTML...)

- [0. Datensatz laden](#0-datensatz-laden)
- [1. Vollständige Schadstoffmessungen und geographische Verteilung](#1-vollständige-schadstoffmessungen-und-geographische-verteilung)
- [2. Clusterberechnung - mehrstufig](#2-clusterberechnung-mehrstufig)
- [3. Clusterbeschreibung mit PCA](#3-clusterbeschreibung-mit-pca)
- [4. Geographische Verteilung der Schadstoffcluster](#4-geographische-verteilung-der-schadstoffcluster)
- [5. Inhaltliche Interpretation](#5-inhaltliche-interpretation)


# 0. Datensatz laden

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import geodatasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
%matplotlib inline

In [ ]:
df = pd.read_csv("data/cleaned_air_quality_data_2025-03-27.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# Liste der Schadstoffvariablen
pollutants = ['Co', 'No2', 'O3', 'Pm10', 'Pm25', 'So2']

# 1. Vollständige Schadstoffmessungen und geographische Verteilung

Um Städte aufgrund ihrer Schadstoffbelastung sinnvoll miteinander vergleichen zu können, muss zuerst ermittelt werden, für welche Städte und Schadstoffe genügend Werte vorliegen.

In [ ]:
# Anteil fehlender Werte pro Spalte
missing_ratios = df[pollutants].isna().mean().sort_values(ascending=True)

print("Anteil fehlender Werte pro Schadstoff:")
print(missing_ratios)


In [ ]:
# Gruppieren: Für jede Stadt den Mittelwert je Schadstoff berechnen
city_pollution_avg = df.groupby('City')[pollutants].mean()

# Ergebnis prüfen
print(city_pollution_avg.head())


In [ ]:
# Städte mit vollständigen Werten (alle 6 Schadstoffe nicht NaN)
city_pollution_complete = city_pollution_avg.dropna()
print(f"Anzahl Städte mit vollständigen Daten: {city_pollution_complete.shape[0]}")


Es liegen also für 404 Städte Werte für alle sechs Schadstoffe vor.

Frage: Wie verteilen sich diese Städte über die geographischen Regionen?

In [ ]:

# Mittelwerte für Koordinaten je Stadt berechnen
coords = df.groupby('City')[['Latitude', 'Longitude']].mean()

# Nur die Städte mit vollständigen Schadstoffdaten behalten
coords_filtered = coords.loc[city_pollution_complete.index]



In [ ]:
# GeoDataFrame mit Punkt-Geometrie
coords_filtered['geometry'] = coords_filtered.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf_complete = gpd.GeoDataFrame(coords_filtered, geometry='geometry', crs='EPSG:4326')


In [ ]:
# Weltkarte laden
world = gpd.read_file(geodatasets.get_path('naturalearth.land'))

# Plot
ax = world.plot(figsize=(12, 6), color='lightgrey', edgecolor='white')
gdf_complete.plot(ax=ax, color='green', markersize=5)

ax.set_title("Städte mit vollständigen Daten für alle sechs Schadstoffe")


Die 404 Städte verteilen sich über alle Kontinente. In dichter besiedelten Regionen liegen erwartungsgemäß mehr Messstationen.

Die ermittelten 404 Städte werden in die folgende Clusteranalyse einbezogen (df = city_pollution_complete).



# 2. Clusterberechnung (mehrstufig)

Die folgende Clusterberechnung erfolgt mehrstufig, da im Prozess immer wieder Extremfälle sichbar werden können, über die im Einzelfall entschieden werden muss.

Damit die Schadstoffwerte, die in unterscheidlichen Einheiten gemessen werden, vergleichbar zu machen, müssen sie zunächst skaliert werden.:

In [ ]:
X = city_pollution_complete.copy()

# Daten skalieren
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# In DataFrame umwandeln für bessere Lesbarkeit
X_scaled_df = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)

print(X_scaled_df.head())


Nun muss eine sinnvolle Anzahl an Clustern ermittelt werden. Dazu wird die Elbow-Methode eingesetzt.

In [ ]:
# Für verschiedene Clusterzahlen die "Inertia" berechnen

inertias = []
k_range = range(1, 31)  # z. B. 1 bis 30 Cluster

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Plot der Elbow-Kurve
plt.figure(figsize=(8, 5))
plt.plot(k_range, inertias, marker='o')
plt.xlabel('Anzahl der Cluster (k)')
plt.ylabel('Inertia')
plt.title('Elbow-Methode zur Bestimmung der optimalen Clusteranzahl')
plt.grid(True)
plt.show()


Die Elbow-Methode liefert kein eindeutiges Ergebnis. Im Datensatz sind mit Sicherheit noch echte Ausreißerstädte enhalten, die das Bild verzerren. Entscheidung: Mit 6 Clustern beginnen.

Als Algorithmus für die Zuordnung der Städte zu Clustern wird KMeans gewählt:

In [ ]:
# KMeans mit 6 Clustern
kmeans = KMeans(n_clusters=6, random_state=42)
labels = kmeans.fit_predict(X_scaled)

# Clusterlabels zum ursprünglichen DataFrame hinzufügen
city_pollution_complete = city_pollution_avg.dropna().copy()
city_pollution_complete['Cluster'] = labels

# Vorschau: Welche Stadt gehört zu welchem Cluster?
print(city_pollution_complete[['Cluster']].value_counts().sort_index())


Hier werden sofort extereme Outlier sichtbar: Cluster 3 und Cluster 5 enthalten jeweils nur eine Stadt.

Welche Städte wurden von KMeans als eigene Cluster bestimmt?

In [ ]:
# Cluster mit nur 1 Stadt ermitteln
einzel_clusters = city_pollution_complete['Cluster'].value_counts()
einzel_clusters = einzel_clusters[einzel_clusters == 1].index.tolist()

# Städte aus diesen Clustern anzeigen
einzelstaedte = city_pollution_complete[city_pollution_complete['Cluster'].isin(einzel_clusters)]

print(einzelstaedte.index.tolist())  # Stadt-Namen


In [ ]:
einzelstaedte

## Städte mit extremen Schadstoffprofilen

### Temuco (Chile)
Temuco fällt durch außergewöhnlich hohe Schwefeldioxid-Werte (SO₂ ≈ 85.6 µg/m³) auf, die in keiner anderen Stadt annähernd erreicht wurden. Diese Belastung ist vermutlich auf die Kombination aus häufiger Holzverbrennung im Winter und der Nähe zum aktiven Vulkan Villarrica zurückzuführen. Auch die PM2.5-Werte lagen mit 67.5 µg/m³ deutlich über dem Mittel.

Aufgrund dieses sehr speziellen, aber durchaus plausiblen Luftschadstoffprofils hätte Temuco ein eigenes Cluster gebildet und dadurch die Vergleichbarkeit der übrigen Gruppen verzerrt. Die Stadt wurde daher aus der Clusteranalyse ausgeschlossen

### Ashkelon (Israel)
Ashkelon wurde aufgrund technischer Auffälligkeiten ausgeschlossen. Der CO-Wert liegt mit durchschnittlich 138.7 µg/m³ mehr als eine Größenordnung über allen anderen Städten im Datensatz. Auch die übrigen Schadstoffwerte wirken inkonsistent. Die stark abweichenden Werte deuten auf eine defekte oder fehlerhafte CO-Messstation hin.

Um eine Verzerrung der Clusteranalyse durch fehlerhafte Daten zu vermeiden, wurde Ashkelon entfernt. Der Ausschluss erfolgte hier nicht, um extreme, aber plausible Umweltbedingungen auszublenden, sondern auf Basis der Datenqualität.

In [ ]:
# Liste der Städte, die aus der Hauptanalyse entfernt werden:
ausreisser = ['Ashkelon', 'Temuco']

# Neue Version des DataFrames ohne diese beiden
city_pollution_cleaned = city_pollution_complete.drop(index=ausreisser)

Nach der Entfernung von Ashkelon und Temuco wird die Clusteranalyse nach demselben Muster auf dem reduzierten Datensatz (df = city_pollution_scaled_cleaned) neu berechnet:

In [ ]:
scaler = StandardScaler()
city_pollution_scaled_cleaned = scaler.fit_transform(city_pollution_cleaned)

In [ ]:
kmeans_cleaned = KMeans(n_clusters=6, random_state=42)
labels_cleaned = kmeans_cleaned.fit_predict(city_pollution_scaled_cleaned)

# Clusterlabels zum DataFrame hinzufügen
city_pollution_cleaned['Cluster'] = labels_cleaned


In [ ]:
city_pollution_cleaned['Cluster'].value_counts().sort_index()


Wieder erscheint ein Cluster mit nur einer Stadt. Das ist nicht ungewöhnlich, da sich durch das Entfernen der extremen Ausreißer nun das gesamte Gefüge verschoben hat und nun auch weniger starke Ausreißer hervortreten können.

In [ ]:
# Cluster mit nur 1 Stadt ermitteln
einzel_clusters = city_pollution_cleaned['Cluster'].value_counts()
einzel_clusters = einzel_clusters[einzel_clusters == 1].index.tolist()

# Städte aus diesen Clustern anzeigen
einzelstaedte = city_pollution_cleaned[city_pollution_cleaned['Cluster'].isin(einzel_clusters)]

print(einzelstaedte.index.tolist())  # Stadt-Namen

In [ ]:
einzelstaedte

### Khorramshahr (Iran)

Khorramshahr wurde aufgrund seines außergewöhnlich hohen Ozon- und Feinstaubniveaus aus der Hauptclusteranalyse ausgeschlossen. Die Werte deuten nicht auf Messfehler, sondern auf eine ernsthafte Luftbelastung hin, die möglicherweise durch regionale Industrieaktivität, hohe Temperaturen und Photochemie verstärkt wird.

Die Stadt bildet aufgrund ihres Extremprofils ein eigenes Cluster, wodurch die übrige Gruppierung verzerrt würde. Daher wird sie separat dokumentiert, aber aus der finalen Clusterstruktur ausgeschlossenn.

In [ ]:
# Liste aktualisieren
ausreisser = ['Ashkelon', 'Temuco', 'Khorramshahr']

# Gefilterter DataFrame
X_cleaned_final = city_pollution_complete.drop(index=ausreisser)

Mit dem nun erhaltenen Datensatz (X_cleaned_final) wird die Clusteranalyse neu berechnet:

In [ ]:
scaler = StandardScaler()
X_scaled_final = scaler.fit_transform(X_cleaned_final)


In [ ]:
kmeans_final = KMeans(n_clusters=6, random_state=42)
labels_final = kmeans_final.fit_predict(X_scaled_final)

# Clusterlabels hinzufügen
X_cleaned_final['Cluster'] = labels_final


In [ ]:
X_cleaned_final['Cluster'].value_counts().sort_index()

Nun haben wir keine zu kleinen Cluster mehr. Die aktuelle Zuordnung wird also nicht weiter verändert.

# 3. Clusterbeschreibung mit PCA

Als nächster Schritt überprüft werden, wie deutlich sich die Cluster in ihren Eigenschaften überschneiden. Dazu wird eine PCA durchgeführt, bei der die sechs Schadstoffe auf zwei Dimensionen reduziert werden, damit sie in einem zweidimensionalen Scatterplot darstellbar sind.

PS: Das hat jetzt nichts mit geografischer Verteilung zu tun, das zeigt nur, wie nah welche Cluster rechnerisch beieinander liegen.

In [ ]:
# PCA auf die skalierten Daten anwenden (X_scaled_final)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled_final)

# In DataFrame mit Clusternummern zusammenführen
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'], index=X_cleaned_final.index)
pca_df['Cluster'] = X_cleaned_final['Cluster']

# Plot definieren
plt.figure(figsize=(10, 6))
for cluster in sorted(pca_df['Cluster'].unique()):
    subset = pca_df[pca_df['Cluster'] == cluster]
    plt.scatter(subset['PC1'], subset['PC2'], label=f'Cluster {cluster}', s=20, alpha=0.7)

plt.title("PCA-Visualisierung der Cluster (2D)")
plt.xlabel("Hauptkomponente 1")
plt.ylabel("Hauptkomponente 2")
plt.legend()
plt.grid(True);


Die PCA-Visualisierung zeigt die Verteilung der Städte im zweidimensionalen Raum, basierend auf ihren durchschnittlichen Schadstoffwerten (CO, NO₂, O₃, PM10, PM2.5, SO₂). Die ursprünglichen sechs Dimensionen wurden mithilfe einer Hauptkomponentenanalyse (PCA) auf zwei Hauptachsen reduziert, die möglichst viel der Varianz im Datensatz abbilden. Die Punkte im Plot repräsentieren einzelne Städte, eingefärbt nach ihrer jeweiligen Clusterzugehörigkeit (KMeans, k=6).

In der linken Hälfte des Plots gruppieren sich die Cluster 1, 3, 4 und 5:

- Cluster 5 erscheint besonders kompakt und dicht, was auf ein sehr homogenes Schadstoffprofil der zugehörigen Städte hindeutet.

- Cluster 1 und Cluster 3 liegen in direkter Nachbarschaft zu Cluster 5 und sind ebenfalls gut erkennbar voneinander abgegrenzt, jedoch weniger dicht. In einem kleinen Bereich überschneiden sich die drei Cluster leicht, was auf gewisse Ähnlichkeiten in den Profilen hindeutet.

- Cluster 4 liegt ebenfalls in der linken Plot-Hälfte, ist jedoch vollständig von den anderen Gruppen abgegrenzt. Die Punktwolke ist weniger dicht, aber klar umrissen, was auf größere interne Varianz, jedoch gute Abgrenzbarkeit hindeutet.

In der rechten Hälfte befinden sich Cluster 0 und Cluster 2:

- Cluster 0 bildet eine erkennbare Gruppe, überscheidet sich aber am Rand mit Cluster 2

- Cluster 2 ist das am stärksten gestreute Cluster und wirkt visuell weniger zusammenhängend. Ein einzelner Punkt liegt sogar deutlich entfernt in der linken Plothälfte, nahe bei Cluster 3. Die Zugehörigkeit zu Cluster 2 wird hier nur durch die algorithmische Einfärbung deutlich.

Die Clusterstruktur ist insgesamt ausreichend gut differenziert und bietet eine fundierte Grundlage für die inhaltliche Analyse der Luftverschmutzungsprofile.

# 4. Geographische Verteilung der Schadstoffcluster

Um zu verstehen, ob und wie Schadstoffmuster mit geographischer Lage zusammenhängen, werden die ermittelten Cluster nun auf eine Weltkarte geplottet.

In [ ]:
# Mittelwerte der Schadstoffe je Cluster
cluster_profiles = X_cleaned_final.groupby('Cluster')[['Co', 'No2', 'O3', 'Pm10', 'Pm25', 'So2']].mean()

# Optional: auf 2 Dezimalstellen runden für bessere Lesbarkeit
cluster_profiles_rounded = cluster_profiles.round(2)

# Anzeigen
print(cluster_profiles_rounded)


In [ ]:
# Koordinaten zuordnen
coords = df.groupby('City')[['Latitude', 'Longitude']].mean()

# Nur die bereinigten Städte (ohne Ashkelon, Temuco, Khorramshahr)
coords_final = coords.loc[X_cleaned_final.index]

# GeoDataFrame bauen
coords_final['Cluster'] = X_cleaned_final['Cluster']
coords_final['geometry'] = coords_final.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf_final = gpd.GeoDataFrame(coords_final, geometry='geometry', crs='EPSG:4326')


In [ ]:
# Farbdefinition individuell, weil die vorgegebenen Farbpaletten visuell schlecht zu unterscheiden waren, und weil man so die zugehörigen Cluster besser zuordnen kann
# Die hier gewählten Farben sollten für Präsentationen evtl. noch überarbeitet werden, weil das Rot_Grün-Problem weiter besteht.

cluster_colors = {
    1: '#4CAF50',  # mittelgrün
    5: '#1B5E20',  # dunkelgrün
    3: '#2196F3',  # mittelblau
    4: '#0D47A1',  # dunkelblau
    0: '#F7A1A1',  # hellrot
    2: '#B71C1C'   # dunkelrot
}

# Manuelle Gruppierung
legend_entries = [
    mpatches.Patch(color='#4CAF50', label='Cluster 1 – Niedrige Belastung (günstiges Klima)'),
    mpatches.Patch(color='#1B5E20', label='Cluster 5 – Niedrige Belastung (urbane Bedingungen)'),
    mpatches.Patch(color='#2196F3', label='Cluster 3 – Mittlere Belastung (Feinstaub)'),
    mpatches.Patch(color='#0D47A1', label='Cluster 4 – Mittlere Belastung (Ozon)'),
    mpatches.Patch(color='#F7A1A1', label='Cluster 0 – Starke Belastung (Feinstaub)'),
    mpatches.Patch(color='#B71C1C', label='Cluster 2 – Starke Belastung (CO und SO₂)')
]


In [ ]:
# Weltkarte
world = gpd.read_file(geodatasets.get_path('naturalearth.land'))

# Plot
fig, ax = plt.subplots(figsize=(14, 8))
world.plot(ax=ax, color='lightgrey', edgecolor='white')

# Punkte pro Cluster plotten
for cluster in sorted(gdf_final['Cluster'].unique()):
    gdf_final[gdf_final['Cluster'] == cluster].plot(
        ax=ax,
        color=cluster_colors[cluster],
        markersize=20,
        label=f'Cluster {cluster}'
    )

# Benutzerdefinierte Legende
ax.legend(handles=legend_entries, title="Clustergruppen", loc='lower left', fontsize=10, title_fontsize=12)

# Titel & Layout
ax.set_title("Städte nach Luftverschmutzungs-Clustern (k=6)", fontsize=15)
ax.set_axis_off()
plt.tight_layout()
plt.show()


# 5. Inhaltliche Interpretation

Abschließend werden die berechneten Cluster inhaltlich interpretiert.

Wir möchten an dieser Stelle daran erinnern, dass die Städte, die den Clustern mit niedriger Belastung zugeordnet wurden, mit Ausnahme von Zürich (Schweiz) alle die von der WHO aktuell empfohlenen Feinstaubwerte übersteigen. Es handelt also nur um *relativ* saubere Luft im Vergleich zu anderen Städten.

## Niedrige Schadstoffbelastung (Cluster 1 und 5)

Cluster 1 und Cluster 5 umfassen beide Städte mit insgesamt niedriger Luftverschmutzung. Dennoch unterscheidet der Algorithmus diese Gruppen aufgrund unterschiedlicher Schadstoffprofile, insbesondere bei Ozon (O₃) und Feinstaub (PM2.5).

Cluster 1 enthält vor allem Städte in Australien, Neuseeland, Kanada, Island und kleineren US-Regionen, oft in klimatisch günstigen, gut belüfteten oder dünn besiedelten Gegenden. Hier sind sowohl Ozon- als auch Feinstaubwerte durchgängig sehr niedrig. Die geringe Ozonbelastung ist besonders auffällig, da man intuitiv in Regionen wie Australien aufgrund des „Ozonlochs“ hohe Ozonwerte erwarten könnte. Tatsächlich bezieht sich das Ozonloch jedoch auf die Stratosphäre – bodennahes Ozon, das als Luftschadstoff wirkt, ist in diesen Städten gering.

Cluster 5 dagegen umfasst eine große Gruppe urbaner Zentren in Ländern wie Japan, Spanien, Frankreich, Deutschland, den USA und Großbritannien. Die Luftqualität ist hier weiterhin vergleichsweise gut, allerdings zeigen sich moderat erhöhte PM2.5- und Ozonwerte. Diese Belastungen lassen sich durch höhere Urbanisierung, Verkehrsdichte, sowie Photochemie in sonnenreichen Regionen erklären – insbesondere bei den Ozonwerten in südeuropäischen und japanischen Städten.

Die Trennung in zwei Cluster innerhalb der „sauberen Städte“ ist daher sachlich sinnvoll und differenzierend:
Cluster 1 steht für „sehr saubere Luft in klimatisch und strukturell begünstigten Regionen“,
Cluster 5 für „gute Luftqualität unter urbanen Rahmenbedingungen“.

In [ ]:
# Schadstoffmittelwerte der beiden Cluster extrahieren, damit man sieht, wo sich die Städte trotz Ähnlichkeit unterscheiden
cluster_1 = cluster_profiles.loc[1]
cluster_5 = cluster_profiles.loc[5]

# Differenz berechnen
diff = (cluster_5 - cluster_1).round(2)
print(diff)

In [ ]:
# Liste der Städte aus demselben Cluster speichern

# Cluster 1-Städte
cluster_1_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 1].index
cluster_1_info = df[df['City'].isin(cluster_1_staedte)][['City', 'Country', 'Population']]
cluster_1_info = cluster_1_info.drop_duplicates(subset='City').set_index('City')
cluster_1_info.to_csv('data/cluster_1_staedte.csv')

# Cluster 5-Städte
cluster_5_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 5].index
cluster_5_info = df[df['City'].isin(cluster_5_staedte)][['City', 'Country', 'Population']]
cluster_5_info = cluster_5_info.drop_duplicates(subset='City').set_index('City')
cluster_5_by_country.to_csv('data/cluster_5_staedte.csv')

## Mittlere Schadstoffbelastung (Cluster 3 und 4)

Cluster 3 und Cluster 4 umfassen Städte mit moderater Luftbelastung, unterscheiden sich jedoch in ihrer dominierenden Schadstoffzusammensetzung und regionalen Verteilung.

Cluster 3 vereint Städte mit leicht erhöhter Feinstaubbelastung (PM2.5, PM10) sowie etwas höheren Werten bei CO und SO₂. Diese Belastungskonstellation weist auf gemischte Emissionsquellen wie Hausbrand, Industrie und Verkehr hin. Geografisch liegen die Städte überwiegend in Süd- und Südostasien, Lateinamerika, der Türkei und Südafrika – also Regionen mit teils unvollständiger Filtertechnik und wachsender Urbanisierung.

Cluster 4 dagegen ist geprägt durch erhöhte Ozon- und NO₂-Werte, bei insgesamt geringerer Partikelbelastung. Diese Konstellation ist typisch für Städte mit starker Verkehrsdichte und intensiver Photochemie, etwa in sonnigen, urbanen Küstenregionen oder dicht besiedelten Stadtstaaten. Cluster 4 umfasst hauptsächlich Städte in Südkorea, Taiwan, China, Mexiko und Südeuropa.

Die Trennung dieser beiden Cluster ist daher gut begründet: Cluster 3 steht für eine diffuse, partikelgetriebene Belastung, während Cluster 4 eher für verkehrsbedingte Ozonbildung und Stickstoffoxidbelastung steht – also unterschiedliche urbane Luftbelastungsmodelle mit verschiedenen Ursachen und Lösungsansätzen.

In [ ]:
# Schadstoffmittelwerte der beiden Cluster extrahieren, damit man sieht, wo sich die Städte trotz Ähnlichkeit unterscheiden
diff_3_4 = (cluster_profiles.loc[3] - cluster_profiles.loc[4]).round(2)
print(diff_3_4)

In [ ]:
# Liste der Städte aus demselben Cluster speichern

# Cluster 3-Städte
cluster_3_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 3].index
cluster_3_info = df[df['City'].isin(cluster_3_staedte)][['City', 'Country', 'Population']]
cluster_3_info = cluster_3_info.drop_duplicates(subset='City').set_index('City')
cluster_3_info.to_csv('data/cluster_3_staedte.csv')

# Cluster 4-Städte
cluster_4_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 4].index
cluster_4_info = df[df['City'].isin(cluster_4_staedte)][['City', 'Country', 'Population']]
cluster_4_info = cluster_4_info.drop_duplicates(subset='City').set_index('City')
cluster_4_info.to_csv('data/cluster_4_staedte.csv')

## Hohe Schadstoffbelastung (Cluster 0 und 2)

Cluster 0 und Cluster 2 umfassen Städte mit insgesamt sehr hoher Luftverschmutzung, unterscheiden sich jedoch deutlich in der Art der dominierenden Schadstoffe.

Cluster 0 ist geprägt von extrem hohen Feinstaubwerten (PM2.5 und PM10), während die Konzentrationen gasförmiger Schadstoffe wie Kohlenmonoxid (CO), Stickstoffdioxid (NO₂) und Schwefeldioxid (SO₂) vergleichsweise niedriger ausfallen. Diese Belastung ist typisch für viele schnell wachsende Megastädte, wie sie in China, Indien und angrenzenden Regionen zu finden sind. Häufige Ursachen sind Hausbrand, industrielle Emissionen, ungünstige Wetterlagen und hohe Bevölkerungsdichte.

Cluster 2 hingegen weist sehr hohe Konzentrationen gasförmiger Luftschadstoffe auf – insbesondere CO und SO₂ –, während die Feinstaubwerte zwar erhöht, aber deutlich geringer sind als in Cluster 0. Die Städte in Cluster 2 liegen überwiegend im Iran und in der Türkei, mit vereinzelten Städten in Israel und Südostasien. Die Belastung in diesen Regionen dürfte vor allem auf verkehrs- und energiebedingte Emissionen, veraltete Infrastruktur und industrielle Verbrennungsprozesse zurückzuführen sein.

Die Trennung dieser beiden Cluster ist daher inhaltlich gut nachvollziehbar: Cluster 0 steht für partikelgetriebene Belastung, Cluster 2 für eine gasförmige Emissionsdominanz. Beide stellen gesundheitlich bedenkliche Situationen dar, aber mit unterschiedlichen Quellenprofilen, die jeweils spezifische Gegenmaßnahmen erfordern würden.

In [ ]:
# Schadstoffmittelwerte der beiden Cluster extrahieren, damit man sieht, wo sich die Städte trotz Ähnlichkeit unterscheiden
diff_0_2 = (cluster_profiles.loc[0] - cluster_profiles.loc[2]).round(2)
print(diff_0_2)

In [ ]:
# Liste der Städte aus demselben Cluster speichern

# Cluster 0-Städte
cluster_0_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 0].index
cluster_0_info = df[df['City'].isin(cluster_0_staedte)][['City', 'Country', 'Population']]
cluster_0_info = cluster_0_info.drop_duplicates(subset='City').set_index('City')
cluster_0_info.to_csv('data/cluster_0_staedte.csv')

# Cluster 2-Städte
cluster_2_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 2].index
cluster_2_info = df[df['City'].isin(cluster_2_staedte)][['City', 'Country', 'Population']]
cluster_2_info = cluster_2_info.drop_duplicates(subset='City').set_index('City')
cluster_2_info.to_csv('data/cluster_2_staedte.csv')

### Übersichtstabelle

Für eine Präsentation ist die Interpretation der Cluster hier noch einmal als Tabelle zusammengefasst:

In [ ]:
# Übersichtstabelle erstellen
data = {
    "Cluster": ["1", "5", "4", "3", "2", "0"],
    "Belastungsprofil": [
        "Sehr saubere Luft",
        "Saubere Luft, urban geprägt",
        "Ozon- & NO₂-betonte Belastung",
        "Partikelbetonte Mischung",
        "Gasdominierte Belastung",
        "Extreme Feinstaubbelastung"
    ],
    "Hauptschadstoffe": [
        "keine dominant",
        "O₃, PM2.5",
        "O₃, NO₂",
        "PM2.5, CO, SO₂",
        "CO, SO₂, NO₂",
        "PM2.5, PM10"
    ],
    "Regionale Tendenz": [
        "Australien, Neuseeland, Kanada",
        "Europa, Japan, USA",
        "Südkorea, Taiwan, Südeuropa",
        "Südostasien, Südafrika, Mexiko",
        "Iran, Türkei, Israel",
        "China, Indien, VAE"
    ],
    "Mögliche Ursachen": [
        "Geringe Emissionen, gute Durchlüftung",
        "Verkehr, Photochemie, urbane Dichte",
        "Verkehr, Sonne, hohe Bevölkerungsdichte",
        "Hausbrand, lokale Industrie, gemischte Quellen",
        "Verkehr, fossile Energie, Industrieabgase",
        "Industrie, Hausbrand, Inversion, Urbanisierung"
    ]
}

df_overview = pd.DataFrame(data)

df_overview